<h1>Section 2.3: Minimization problems</h1>

In this worksheet we solve a juice drink LP and the Foraging Herbavore LP with the simplex method.

In [0]:
import numpy as np
import pandas as pd

def SwapRow(arrayA,rowi,rowj): #interchange rows i and j
    arrayB=arrayA.copy()
    arrayB[rowi]=arrayA[rowj]
    arrayB[rowj]=arrayA[rowi]
    return arrayB
def AddRow(arrayA,rowi,rowj,s): #add s * row j to row i
    arrayB=arrayA.copy()
    arrayB=arrayB.astype(float) #to avoid division errors with integer arrays
    arrayB[rowi]=arrayA[rowi]+s*arrayA[rowj]
    return arrayB
def MultiplyRow(arrayA,rowi,s): #multiply row i by s
    arrayB=arrayA.copy()
    arrayB=arrayB.astype(float) #to avoid division errors with integer arrays
    arrayB[rowi]=s*arrayA[rowi]
    return arrayB
def Pivot(arrayA,rowi,colj): #pivot matrix at row i and column j
    arrayB=arrayA.copy()
    arrayB=MultiplyRow(arrayA,rowi,1/arrayA[rowi,colj])
    for x in range(0,rowi):
        arrayB=AddRow(arrayB,x,rowi,-arrayB[x,colj])
    for x in range(rowi+1,arrayB.shape[0]):
        arrayB=AddRow(arrayB,x,rowi,-arrayB[x,colj])
    return arrayB

The next code block has the data for the LPs.  The foraging herbavore data is commented out.  If you edit this for different problems, you need to put all the constraints which will have slack variables first, then the ones that will have excess.

In [0]:
c = np.array([2,10])
#c = np.array([45.55, 21.87])
#create a one-dimensional array of objective function coefficients

A = np.array([[0.5,0.25],
             [1,3],
             [1,1]])
#A = np.array([[1.64,2.67],
#             [2.11,2.3]])
#create a two-dimensional array of constraint coefficients

b = np.array([4,20,12])
#b = np.array([31.2,13.9])
#create a one-dimensional array of constraint bounds

slack_variables = [1]
excess_variables = [2]
artificial_variables = [2,3]
#slack_variables = [1]
#excess_variables = [2]
#artificial_variables = [2]
#give a list of each type of variable.

M = 100
#give value of M for big M method

In [0]:
n=len(c)
m=len(b)
#find the number of decision variables and constraints, respectively
x=['x_'+str(i+1) for i in range(n)]
s=['s_'+str(i) for i in slack_variables]
e=['e_'+str(i) for i in excess_variables]
a=['a_'+str(i) for i in artificial_variables]
Labels=['z']+x+s+e+a+['RHS']
#create labels for variables
def Tableau(M):
    return pd.DataFrame(M,columns=Labels)
#procedure to add column labels to an LPMatrix

var_block = np.zeros((m,len(s)+len(e)+len(a)))
for i in slack_variables:
  var_block[i-1,i-1]=1
for i in excess_variables:
  var_block[i-1,i-1]=-1
for i in artificial_variables:
  var_block[i-1,i-1+len(e)]=1
#create coefficients for slack, excess, artificial variables

LPMatrix = np.block([
    [1,-c,np.zeros(len(s)+len(e)),np.full(len(a),-M),0],
    [np.zeros((m,1)),A,var_block,b[:,np.newaxis]]
])
#create array corresponding to the initial tableau
def RowRatios(M,c):
    for i in range(m):
        if (M[i+1,c]>0.001):
            print("Row", i+1, "Ratio =", M[i+1,-1]/M[i+1,c])
        else:
            print("Row", i+1, "Ratio Undefined")
#row ratio test for LPMatrix M and column c
def Iterate(M,r,c):
    M2=MultiplyRow(M,r,1/M[r,c])
    M2=Pivot(M2,r,c)
    return M2

In [0]:
M1=LPMatrix
Tableau(M1)

,z,x_1,x_2,s_1,e_2,a_2,a_3,RHS
0,1.0,-2.0,-10.00,0.0,0.0,-100.0,-100.0,0.0
1,0.0,0.5,0.25,1.0,0.0,0.0,0.0,4.0
2,0.0,1.0,3.00,0.0,-1.0,1.0,0.0,20.0
3,0.0,1.0,1.00,0.0,0.0,0.0,1.0,12.0


First we pivot so that our artificial variables are basic in our initial tableau.  Recall that we use <code>Iterate(tableau,row,column).</code>

In [0]:
M1=Iterate(M1,2,5)
M1=Iterate(M1,3,6)
Tableau(M1)

Now we carry out the simplex algorithm for minimization.

In [0]:
RowRatios(M1,2)

In [0]:
M2=Iterate(M1,2,2)
Tableau(M2)

In [0]:
RowRatios(M2,1)

In [0]:
M3=Iterate(M2,1,1)
Tableau(M3)

In [0]:
RowRatios(M3,4)

In [0]:
M4=Iterate(M3,3,4)
Tableau(M4)